In [154]:
import requests, lxml, cchardet, re
from bs4 import BeautifulSoup


Picked 'user-agent' for my browser from https://www.whatismybrowser.com/detect/what-is-my-user-agent/



In [3]:
url = "https://patient.info/forums/discuss/browse/depression-683"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    #Switch the User-agent to avoid anti-scraping measures and "make believe" this is a real human requests those URLs
}
session_object = requests.Session() 
#Update : use a session to avoid making one request per page + 
#          saving time& the server's ressources
response = session_object.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'lxml') #using lxml to parse the main page, saving time

# print(soup)

In [4]:
from lxml import etree

#### This part is only for testing purposes

In [5]:
#dom = etree.HTML(str(soup)) #turning the parsed HTML into a dom.xpath object that can be selected using xPath

#for j in range(1, 37):
#try:
#        print('https://patient.info'+dom.xpath(f'/html/body/div/div/div[2]/div[1]/div/div/div/div[1]/div/ul/li[{j}]/div/div[2]/article/h3/a/@href')[0])
#    except:
#        print(dom.xpath(f'/html/body/div/div/div[2]/div[1]/div/div/div/div[1]/div/ul/li[{j}]/div/div[2]/article/h3/a/@href'), "couldn't be found")
#        continue

In [11]:
### LOOP THROUGH EACH PAGE FROM 1 TO 198 starting with : 
### https://patient.info/forums/discuss/browse/depression-683?page=0#group-discussions LOOP AROUND PAGE = 0 
### then PAGE = 1 then PAGE = 2...up to 197
#import logging 

post_urls = []
post_url = 0

for i in range(): 
    
    #USE len(198) in range() for a FULL crawl. Do not crawl the entire forum unless necessary. 
    #The final results are on github. If you want to test the code, simply use a range(1,2)
    
    page_url = f"https://patient.info/forums/discuss/browse/depression-683?page={i}#group-discussions"
    response = session_object.get(page_url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')
 ### FOR EACH TOPIC NAME (use XPATH) GRAB URL AND APPEND TO A LIST   
    for j in range(1, 37):
        XPATH = f"/html/body/div/div/div[2]/div[1]/div/div/div/div[1]/div/ul/li[{j}]/div/div[2]/article/h3/a/@href"
        
        dom = etree.HTML(str(soup))
        try:
            post_url = 'https://patient.info' + dom.xpath(XPATH)[0]
            post_urls.append(post_url)
        except:
            #logging.exception("An exception was thrown!")
            #print(i, "i", page_url)
            #print(post_url, "couldn't be scraped")
            
            continue


In [16]:
post_urls[-5:]

['https://patient.info/forums/discuss/depression-fake-illnes--11369',
 'https://patient.info/forums/discuss/citralpram-8895',
 'https://patient.info/forums/discuss/i-suffered-with-postnatal-depression-after-the-birth-of--4901',
 'https://patient.info/forums/discuss/i-d-like-to-hear-from-anyone-with-similar-experience-to--6919',
 'https://patient.info/forums/discuss/i-suffer-with-panic-attacks-since-taking-dothapax-i-have--838']

In [174]:
### THEN LOOP THROUGH EVERY URL IN LIST, REQUEST URL, AND SCRAP MAIN POST'S TEXT

XPATH = f'/html/body/div/div/div[2]/div[1]/div/div/div/div[1]/article/div[3]/input'
post_dict = []
fieldnames = ["URL","Main_Post"]

for i in range(): 
    #USE len(post_urls) in range() for a FULL crawl. Do not crawl the entire URL list unless necessary. 
    #The final results are on github. If you want to test the code, simply use a range(1,5)

    response = session_object.get(post_urls[i], headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')
    website = etree.HTML(str(soup))
    try:
        e = website.xpath('.//*[@id="post_content"]/input')[0]
        text = e.attrib["value"]
        text = re.sub(r"\r", "", text)
        text = re.sub(r"\n", "", text)
        text = re.sub(r"<p>", "", text)
        text = re.sub(r"</p>", "", text)
        post_dict.append({"URL":post_urls[i], "Main_Post":text})
    except: 
        continue

In [176]:
import csv
  
with open('Results.csv', 'w', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames,lineterminator = '\n', delimiter = ";")
    writer.writeheader()
    writer.writerows(post_dict)